# Práce s kamerou Basler v Jupyter Notebooku
<img src='images/basler-logo.jpg' style="width:50%">

Vzhledem k tomu, že software Pylon je nejen vlastní program pro připojení a manipulaci s kamerami, ale zároveň SDK, budeme používat přímo SDK. SDK je psáno v C, C++ nebo C#. Basler k tomu však v roce 2018 vytvořil wrapper do Pythonu s názvem `pypylon`.

My jsme k tomu vytvořili rozhraní s názvem `pypylon_opencv_viewer`. To umožňuje se připojit ke konkrétní kameře, zobrazit si snímky z kamery a dále nastavovat některé parametry kamery.

## Nastavení
Import potřebných knihoven

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cv2
import yaml
from pypylon import pylon 

from IPython.display import Image, display, clear_output

%matplotlib inline

Úvodni nastavení parametrů kamery. Je potřeba dopsat správné **sériové číslo** vlastní kamery. 

V případě, že se nastavuje cesta k ukládání fotek, je třeba nahradit veškerá `\` za `\\`. Při defaultním nastavení se fotky ukládají do složky s tímto ipynb.

In [2]:
#serial_number = '22724646' ### sériové číslo řádkové kamery Ra
#serial_number = '21991061' ### sériové číslo maticové kamery Ac RGB
serial_number = '22018113' ### sériové číslo maticové kamery Ac Mono
width = 1280
height = 1080
image_folder = '.' ### defaultní nastavení složky pro ukládání snímků

Připojení kamery pomocí `pypylonu`. Je zapotřebí zkontrolovat sériové číslo kamery.

In [3]:
# Pypylon gets camera by serial number
info = None
for i in pylon.TlFactory.GetInstance().EnumerateDevices():
    if i.GetSerialNumber() == serial_number:
        info = i
        break
else:
    print('Camera with {} serial number not found'.format(serial_number))

# VERY IMPORTANT STEP! To use Basler PyPylon OpenCV viewer you have to call .Open() method on you camera
if info is not None:
    camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateDevice(info)) 
    camera.Open()

Pomocná funkce pro zobrazování obrázků

In [4]:
def show_image(img):
    window_name = 'Camera capture (' + serial_number + ')'
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
    cv2.resizeWindow(window_name, width, height)
    cv2.imshow(window_name, img)

Vytvoření GUI widgetů pro manipulaci s parametry kamery. Zásady pro tvorbu JSON notace widgetů vizte na [link](https://github.com/mbalatsko/pypylon-opencv-viewer#list-of-features).

In [5]:
# Configuration PyPylon Viewer to load features for RGB Matrix Camera
VIEWER_CONFIG_RGB_MATRIX = {
    "features": [
        {
            "name": "Height",
            "type": "int",
            "value": 1080,
            "unit": "px",
            "step": 2,
        },
        {
            "name": "Width",
            "type": "int",
            "value": 1920,
            "unit": "px",
            "step": 2,
        },
        {
            "name": "CenterX",
            "type": "bool",
        },
        {
            "name": "CenterY",
            "type": "bool",

        },
        {
            "name": "OffsetX",
            "type": "int",
            "dependency": {"CenterX": False},
            "unit": "px",
            "step": 2,
        },
        {
            "name": "OffsetY",
            "type": "int",
            "dependency": {"CenterY": False},
            "unit": "px",
            "step": 2,
        },
        {
            "name": "AcquisitionFrameRateAbs",
            "type": "int",
            "unit": "fps",
            "max": 150,
            "min": 1,
        },
        {
            "name": "ExposureAuto",
            "type": "choice_text",
            "options": ["Off", "Once", "Continuous"],
            "style": {"button_width": "90px"}
        },
        {
            "name": "ExposureTimeAbs",
            "type": "int",
            "dependency": {"ExposureAuto": "Off"},
            "unit": "μs",
            "step": 100,
            "max": 35000,
            "min": 500,
        },
        {
            "name": "BalanceWhiteAuto",
            "type": "choice_text",
            "options": ["Off", "Once", "Continuous"],
            "style": {"button_width": "90px"}
        },
    ],
    "features_layout": [
        ("Height", "Width"), 
        ("OffsetX", "CenterX"), 
        ("OffsetY", "CenterY"), 
        ("ExposureAuto", "ExposureTimeAbs")
    ],
    "actions_layout": [
        ("StatusLabel"),
        ("SaveConfig", "LoadConfig", "ContinuousShot", "SingleShot"), 
        ("UserSet")
    ],
    "default_user_set": "UserSet3",
}


In [6]:
# Configuration PyPylon Viewer to load features for RGB Matrix Camera
VIEWER_CONFIG_MONO_MATRIX = {
    "features": [
         {
            "name": "GainRaw",
            "type": "int",
            "step": 1,
        },
        {
            "name": "Height",
            "type": "int",
            "unit": "px",
            "step": 2,
        },
        {
            "name": "Width",
            "type": "int",
            "unit": "px",
            "step": 2,
        },
        {
            "name": "CenterX",
            "type": "bool",
        },
        {
            "name": "CenterY",
            "type": "bool",

        },
        {
            "name": "OffsetX",
            "type": "int",
            "dependency": {"CenterX": False},
            "unit": "px",
            "step": 2,
        },
        {
            "name": "OffsetY",
            "type": "int",
            "dependency": {"CenterY": False},
            "unit": "px",
            "step": 2,
        },
        {
            "name": "AcquisitionFrameRateAbs",
            "type": "int",
            "unit": "fps",
            "dependency": {"AcquisitionFrameRateEnable": True},
            "max": 150,
            "min": 1,
        },
        {
            "name": "AcquisitionFrameRateEnable",
            "type": "bool",
        },
        {
            "name": "ExposureAuto",
            "type": "choice_text",
            "options": ["Off", "Once", "Continuous"],
            "style": {"button_width": "90px"}
        },
        {
            "name": "ExposureTimeAbs",
            "type": "int",
            "dependency": {"ExposureAuto": "Off"},
            "unit": "μs",
            "step": 500,
            "max": 35000,
            "min": 500,
        },
    ],
    "features_layout": [
        ("Height", "Width"), 
        ("OffsetX", "CenterX"), 
        ("OffsetY", "CenterY"), 
        ("ExposureTimeAbs", "ExposureAuto"),
        ("AcquisitionFrameRateAbs", "AcquisitionFrameRateEnable"),
        ("GainRaw")
    ],
    "actions_layout": [
        ("StatusLabel"),
        ("SaveConfig", "LoadConfig", "ContinuousShot", "SingleShot"), 
        ("UserSet")
    ],
    "default_user_set": "UserSet3",
}


In [7]:
# Configuration PyPylon Viewer to load features for Monochromatic Line Scan Camera
VIEWER_CONFIG_MONO_LINE = {
    "features": [
         {
            "name": "GainRaw",
            "type": "int",
            "step": 1,
        },
        {
            "name": "Height",
            "type": "int",
            "unit": "px",
            "step": 2,
        },
        {
            "name": "Width",
            "type": "int",
            "unit": "px",
            "step": 2,
        },
        {
            "name": "CenterX",
            "type": "bool",
        },
        {
            "name": "OffsetX",
            "type": "int",
            "dependency": {"CenterX": False},
            "unit": "px",
            "step": 2,
        },
        {
            "name": "AcquisitionLineRateAbs",
            "type": "int",
            "max": 5000,
            "min": 100,
            "step": 100,
        },
        {
            "name": "ExposureAuto",
            "type": "choice_text",
            "options": ["Off", "Once", "Continuous"],
            "style": {"button_width": "90px"}
        },
        {
            "name": "ExposureTimeAbs",
            "type": "int",
            "dependency": {"ExposureAuto": "Off"},
            "unit": "μs",
            "step": 500,
            "max": 35000,
            "min": 500,
        },
    ],
    "features_layout": [
        ("Height", "Width"), 
        ("OffsetX", "CenterX"), 
        ("ExposureTimeAbs", "ExposureAuto"),
        ("AcquisitionLineRateAbs", "GainRaw")
    ],
    "actions_layout": [
        ("StatusLabel"),
        ("SaveConfig", "LoadConfig", "ContinuousShot", "SingleShot"), 
        ("UserSet")
    ],
    "default_user_set": "UserSet3",
}


Funkce pro image processing. Zatím pouze zobrazuje obrázky.

In [8]:
def impro(img):
    show_image(img)
    return img

## Main
Výkonný kus kódu využívající vlastní knihovnu pro práci s kamerou a knihovnou OpenCV. Umožňuje nastavení parametrů pomocí GUI widgetů.
- Pro spuštění okna kamery je zapotřebí kliknout na tlačítko `Continuous shot` (pro získání streamu) nebo `Single shot` (pro jeden snímek) 
- Okno kamery se vypne stisknutím tlačítka `q`.
- Obrázek se uloží stisknutím tlačítka `s`.
- Pro projevení změn v nastavení pomocí GUI prvků je nutné ukončit okno kamery a znovu ho spustit.

In [9]:
import sys
sys.path
sys.path.insert(0, '/home/kasape/Documents/pypylon-opencv-viewer/')

In [10]:
from pypylon_opencv_viewer import BaslerOpenCVViewer
viewer = BaslerOpenCVViewer(camera)
viewer.set_configuration(VIEWER_CONFIG_MONO_MATRIX)
#viewer.set_impro_function(impro)
viewer.show_interactive_panel(window_size=(width, height), image_folder=image_folder)

Output()